## ランダムフォレスト　デフォルト

In [1]:
import numpy as np
import pandas as pd

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error, confusion_matrix 
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.model_selection import train_test_split

from sklearn import tree


In [15]:
# pickle ファイルから読み込み
train_pkl = pd.read_pickle('./train.pk2')

In [16]:
train_pkl.shape

(242150, 83)

In [17]:
train_pkl.describe()

,loan_amnt,term,interest_rate,employment_length,credit_score,application_type,application_type + credit_score,application_type + employment_length,application_type + interest_rate,application_type + loan_amnt,...,credit_score - loan_amnt,credit_score - term,employment_length - interest_rate,employment_length - loan_amnt,employment_length - term,interest_rate - loan_amnt,interest_rate - term,loan_amnt - term,loan_status,id
count,242150.000000,242150.000000,242150.000000,242150.000000,242150.000000,242150.000000,242150.000000,242150.000000,242150.000000,242150.000000,...,242150.000000,242150.000000,242150.000000,242150.000000,242150.000000,242150.000000,242150.000000,242150.000000,242150.000000,242150.00000
mean,1520.389009,3.431658,13.801496,6.672298,683.575024,0.007359,683.582383,6.679657,13.808855,1520.396369,...,-836.813985,680.143366,-7.129197,-1513.716711,3.240640,-1506.587514,10.369838,1516.957351,0.174916,121074.50000
std,830.250197,0.822794,4.588924,3.657411,29.554795,0.085469,29.556557,3.656696,4.590767,830.254507,...,827.868342,29.590202,5.864958,830.014131,3.725264,829.295029,4.269758,829.966347,0.379896,69902.82818
min,323.797279,3.000000,5.704849,0.000000,655.424269,0.000000,655.424269,0.000000,5.704849,323.797279,...,-3175.687333,650.510945,-27.528590,-3851.867974,-5.000000,-3839.018030,0.812533,320.797279,0.000000,0.00000
25%,761.954545,3.000000,10.876086,3.000000,659.531106,0.000000,659.535793,3.000000,10.881283,761.954545,...,-1456.738754,656.161458,-11.415029,-2145.168102,0.000000,-2137.691382,7.722855,758.919295,0.000000,60537.25000
50%,1212.680586,3.000000,13.543833,9.000000,678.672563,0.000000,678.678756,9.000000,13.547887,1212.686834,...,-543.566096,675.292875,-6.997864,-1206.503023,5.000000,-1198.974968,10.279170,1209.357969,0.000000,121074.50000
75%,2152.213330,3.000000,17.172395,10.000000,698.591960,0.000000,698.595488,10.000000,17.176818,2152.223840,...,-94.138528,695.267101,-3.048240,-756.153564,7.000000,-748.529252,12.980316,2148.261662,0.000000,181611.75000
max,3851.867974,5.000000,27.980604,10.000000,808.551641,1.000000,808.551641,11.000000,28.980604,3851.867974,...,462.497412,804.433596,4.295151,-313.797279,7.000000,-316.593872,24.260526,3846.867974,1.000000,242149.00000


In [18]:
# object 型削除
# train_pkl.drop(['grade', 'purpose'], axis=1, inplace=True)

In [19]:
# train_pkl.info()
train_pkl.shape

(242150, 83)

## 訓練データとテストデータに分割

In [20]:
# 訓練とテストデータに分割
train, test = train_test_split(train_pkl, test_size=0.2, random_state=42)

In [21]:
# ターゲットと特徴量の分割
train_X = train.iloc[:, :-2]
train_y = train.loan_status

In [22]:
train_X.shape, train_y.shape

((193720, 81), (193720,))

## 訓練データで訓練

In [23]:
RF = RandomForestClassifier(random_state=42)

In [24]:
RF = RF.fit(train_X, train_y)

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [18]:
RF.feature_importances_

array([0.29278603, 0.01597332, 0.34905624, 0.03817578, 0.30227264,
       0.001736  ])

In [20]:
sorted(
    zip(map(lambda x: round(x, 3), RF.feature_importances_), train_X.columns),
    reverse=True)

[(0.349, 'interest_rate'),
 (0.302, 'credit_score'),
 (0.293, 'loan_amnt'),
 (0.038, 'employment_length'),
 (0.016, 'term'),
 (0.002, 'application_type')]

In [21]:
# 訓練済みの決定木を視覚化
# dot_data = tree.export_graphviz(RF, out_file=None,
#                                feature_names=train.iloc[:, 1:].columns,
#                                class_names=train.Survived.name,
#                                rounded=True,
#                                filled=True,
#                                special_characters=True)

## テストデータで実行

In [23]:
# ターゲットと特徴量の分割
test_x = test.iloc[:, 1:-1]
test_y = test.loan_status

In [24]:
test_x.shape, test_y.shape

((48430, 6), (48430,))

In [75]:
train_y.sum() / train_y.shape[0], test_y.sum() / test_y.shape[0]

(0.17461284327895932, 0.17613049762543878)

In [25]:
pred_y = RF.predict(test_x)

In [26]:
pred_y.shape

(48430,)

In [70]:
test_y.sum(), pred_y.sum()

(8530, 2017)

In [28]:
confusion_matrix(test_y, pred_y)

array([[38804,  1096],
       [ 7609,   921]], dtype=int64)

In [29]:
accuracy_score(test_y, pred_y)

0.8202560396448483

In [30]:
f1_score(test_y, pred_y)

0.1746468190006637

In [66]:
recall_score(test_y, pred_y)

0.10797186400937867

In [71]:
confusion_matrix(test_y, pred_y)[1, 1] / (confusion_matrix(test_y, pred_y)[1, 1] + confusion_matrix(test_y, pred_y)[1, 0])

0.10797186400937867

In [65]:
precision_score(test_y, pred_y)

0.45661874070401587

In [72]:
confusion_matrix(test_y, pred_y)[1, 1] / (confusion_matrix(test_y, pred_y)[1, 1] + confusion_matrix(test_y, pred_y)[0, 1])

0.45661874070401587

In [68]:
2 / ((1 / precision_score(test_y, pred_y)) + (1 / recall_score(test_y, pred_y)))

0.1746468190006637

## 検証データで実行

In [31]:
# 検証データ読み込み
valid = pd.read_pickle('./test.pk1')

In [32]:
valid.shape

(26900, 9)

In [33]:
# ID の保存
valid_pass = valid.id.values

In [34]:
valid_X = valid.copy()

In [38]:
valid_X.drop(['id', 'grade', 'purpose'], axis=1, inplace=True)

In [39]:
valid_X.describe()

,loan_amnt,term,interest_rate,employment_length,credit_score,application_type
count,26900.000000,26900.000000,26900.000000,26900.000000,26900.000000,26900.000000
mean,1515.095458,3.433309,13.778459,6.700669,683.840319,0.007175
std,827.710449,0.823945,4.587711,3.637777,29.685781,0.084401
min,353.090088,3.000000,5.745858,0.000000,655.435675,0.000000
25%,753.766957,3.000000,10.846102,3.000000,659.589762,0.000000
50%,1213.754480,3.000000,13.551492,9.000000,678.707870,0.000000
75%,2147.699462,3.000000,17.128929,10.000000,698.684676,0.000000
max,3812.969623,5.000000,27.503147,10.000000,811.676356,1.000000


In [40]:
valid_X.shape, train_X.shape

((26900, 6), (193720, 6))

In [41]:
pred_valid_y = RF.predict(valid_X)

In [56]:
valid_pass.shape, pred_valid_y.shape

((26900,), (26900,))

In [52]:
type(valid_pass), type(pred_valid_y)

(numpy.ndarray, numpy.ndarray)

In [57]:
result_df = pd.DataFrame(pred_valid_y, valid_pass, columns=['result'])

In [58]:
result_df.head()

,result
242150,0
242151,0
242152,0
242153,0
242154,0


In [59]:
result_df.to_csv("./RF_1.csv", header=False)